###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2017 L.A. Barba, N.C. Clementi

# Catch things in motion

This module of the _Engineering Computations_ course is our launching pad to investigate _change_, _motion_, _dynamics_, using computational thinking, Python, and Jupyter.

The foundation of physics and engineering is the subject of **mechanics**: how things move around, when pushed around. Or pulled… in the beginning of the history of mechanics, Galileo and Newton seeked to understand how and why objects fall under the pull of gravity.

This first lesson will explore motion by analyzing images and video, to learn about velocity and acceleration.

## Acceleration of a falling ball

Let's start at the beginning. Suppose you want to use video capture of a falling ball to _compute_ the acceleration of gravity. Could you do it? With Python, of course you can!

Here is a neat video we found online, produced over at MIT several years ago [1]. It shows a ball being dropped in front of a metered panel, while lit by a stroboscopic light. Watch the video!

In [ ]:
from IPython.display import YouTubeVideo
vid = YouTubeVideo("xQ4znShlK5A")
display(vid)

We learn on the video that the marks on the panel are every $0.25\rm{m}$, and on the [website](http://techtv.mit.edu/collections/physicsdemos/videos/831-strobe-of-a-falling-ball) they say that the strobe light flashes at about 15 Hz (that's 15 times per second). The final [image on Flickr](https://www.flickr.com/photos/physicsdemos/3174207211), however, notes that the strobe fired 16.8 times per second. So we have some uncertainty already!

Luckily, the MIT team obtained one frame with the ball visible at several positions as it falls. This, thanks to the strobe light and a long-enough exposure of that frame. What we'd like to do is use that frame to capture the ball positions digitally, and then obtain the velocity and acceleration from the distance over time. 

You can find several toolkits for handling images and video with Python; we'll start with a simple one called [`imageio`](https://imageio.github.io). Import this library like any other, and let's load `numpy` and `pyplot` while we're at it.


In [ ]:
import imageio
import numpy
from matplotlib import pyplot

### Read the video

With the `get_reader()` method of `imageio`, you can read a video from its source into a _Reader_ object. You don't need to worry too much about the technicalities here—we'll walk you through it all—but check the type, the length (for a video, that's number of frames), and notice you can get info, like the frames-per-second, using `get_meta_data()`.

In [ ]:
reader = imageio.get_reader('http://techtv.mit.edu/videos/831-strobe-of-a-falling-ball/download.mp4')

In [ ]:
type(reader)

In [ ]:
len(reader)

In [ ]:
fps = reader.get_meta_data()['fps']
print(fps)

##### Note:

You may get this error after calling `get_reader()`:

NeedDownloadError: Need ffmpeg exe. You can obtain it with either:
  - install using conda: `conda install ffmpeg -c conda-forge`
  - download by calling: `imageio.plugins.ffmpeg.download()`

If you do, follow the tips to install the needed `ffmpeg` tool.

### Show a video frame in an interactive figure

With `imageio`, you can grab one frame of the video, and then use `pyplot` to show it as an image. But we want to interact with the image, somehow.

So far in this course, we have used the command `%matplotlib inline` to get our plots rendered _inline_ in a Jupyter notebook. There is an alternative command that gives you some interactivity on the figures: `%matplotlib notebook`. Execute this now, and you'll see what it does below, when you show the image in a new figure.

In [ ]:
%matplotlib notebook

Now we can use the `get_data()` method on the `imageio` _Reader_ object, to grab one of the video frames, passing the frame number. Below, we use it to grab frame number 1100, and then print the `shape` attribute to see that it's an "array-like" object with three dimensions: they are the pixel numbers in the horizontal and vertical directions, and the number of colors (3 colors in RGB format). Check the type to see that it's an `imageio` _Image_ object.

In [ ]:
image = reader.get_data(1100)
image.shape

In [ ]:
type(image)

Naturally, `imageio` plays well with `pyplot`. You can use [`pyplot.imshow()`](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.imshow.html) to show the image in a figure. We chose to show frame 1100 after playing around a bit and finding that it gives a good view of the long-exposure image of the falling ball.

##### Explore:

Check out the neat interactive options that we get with `%matplotlib notebook`. Then go back and change the frame number above, and show it below. Notice that you can read $(x,y)$ coordinates of your cursor tip while you hover on the image with the mouse.

In [ ]:
pyplot.imshow(image, interpolation='nearest');

### Capture mouse clicks on the frame

Okay! Here is where things get really interesting. Matplotlib has the ability to create [event connections](https://matplotlib.org/devdocs/users/event_handling.html?highlight=mpl_connect), that is, connect the figure canvas to user-interface events on it, like mouse clicks. 

To use this ability, you write a function with the events you want to capture, and then connect this function to the Matplotlib "event manager" using [`mpl_connect()`](https://matplotlib.org/devdocs/api/backend_bases_api.html#matplotlib.backend_bases.FigureCanvasBase.mpl_connect). In this case, we connect the `'button_press_event'` to the function named `onclick()`, which captures the $(x,y)$ coordinates of the mouse click on the figure. Magic!

In [ ]:
fig = pyplot.figure()

pyplot.imshow(image, interpolation='nearest')

coords = []
def onclick(event):
    '''Capture the x,y coordinates of a mouse click on the image'''
    ix, iy = event.xdata, event.ydata
    coords.append([ix, iy]) 

connectId = fig.canvas.mpl_connect('button_press_event', onclick)


Notice that in the code cell above, we created an empty list named `coords`, and inside the `onclick()` function, we are appending to it the $(x,y)$ coordinates of each mouse click on the figure. After executing the cell above, you have a connection to the figure, via the user interface: try clicking with your mouse on the endpoints of the white lines of the metered panel (click on the edge of the panel to get approximately equal $x$ coordinates), then print the contents of the `coords` list below.

In [ ]:
coords

The $x$ coordinates are pretty close, but there is some variation due to our shaky hand (or bad eyesight), and perhaps because the metered panel is not perfectly vertical. We can cast the `coords` list to a NumPy array, then grab all the first elements of the coordinate pairs, then get the standard deviation as an indication of our error in the mouse-click captures.

In [ ]:
numpy.array(coords)[:,0]

In [ ]:
numpy.array(coords)[:,0].std()

Depending how shaky _your_ hand was, you may get a different value, but we got a standard deviation of about one pixel. Pretty good!

Now, let's grab all the second elements of the coordinate pairs, corresponding to the $y$ coordinates, i.e., the vertical positions of the white lines on the video frame.

In [ ]:
y_lines = numpy.array(coords)[:,1]
y_lines

Looking ahead, what we'll do is repeat the process of capturing mouse clicks on the image, but clicking on the ball positions. Then, we will want to have the vertical positions converted to physical length (in meters), from the pixel numbers on the image.

You can get the scaling from pixels to meters via the distance between two white lines on the metered panel, which we know is $0.25\rm{m}$. 

Let's get the average vertical distance between two while lines, which we can calculate as:

\begin{equation}
\overline{\Delta y} = \sum_{i=0}^N \frac{y_{i+1}-y_i}{N-1}
\end{equation}

In [ ]:
gap_lines = y_lines[1:] - y_lines[0:-1]
gap_lines.mean()

##### Discuss with your neighbor

* Why did we slice the `y_lines` array like that? If you can't explain it, write out the first few terms of the sum above and think!

### Compute the acceleration of gravity

We're making good progress! You'll repeat the process of showing the image on an interactive figure, and capturing the mouse clicks on the figure canvas: but this time, you'll click on the ball positions. 

Using the vertical displacements of the ball, $\Delta y_i$, and the known time between two flashes of the strobe light, $1/16.8\rm{s}$, you can get the velocity and acceleration of the ball! But first, to convert the vertical displacements to meters, you'll multiply by $0.25\rm{m}$ and divide by `gap_lines.mean()`.

Before clicking on the ball positions, you may want to inspect the high-resolution final [photograph on Flickr](https://www.flickr.com/photos/physicsdemos/3174207211)—notice that the first faint image of the falling ball is just "touching" the ring finger of Bill's hand. We decided _not_ to use that photograph in our lesson because the Flickr post says _"All rights reserved"_, while the video says specifically that it is licensed under a Creative Commons license. In other words, MIT has granted permission to use the video, but _not_ the photograph. _Sigh_.

OK. Go for it: capture the clicks on the ball!

In [ ]:
fig = pyplot.figure()

pyplot.imshow(image, interpolation='nearest')

coords = []
def onclick(event):
    '''Capture the x,y coordinates of a mouse click on the image'''
    ix, iy = event.xdata, event.ydata
    coords.append([ix, iy]) 

connectId = fig.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
coords

We'll scale the vertical displacements of the falling ball as explained above (to get distance in meters), then use the known time between flashes of the strobe light, $1/16.8\rm{s}$, to compute estimates of the velocity and acceleration of the ball, using:

\begin{equation}
v_i = \frac{y_{i+1}-y_i}{\Delta t}, \qquad a_i = \frac{v_{i+1}-v_i}{\Delta t}
\end{equation}


In [ ]:
y_coords = numpy.array(coords)[:,1]
delta_y = (y_coords[1:] - y_coords[:-1]) *0.25 / gap_lines.mean()

In [ ]:
v = delta_y * 16.8
v

In [ ]:
a = (v[1:] - v[:-1]) *16.8
a

In [ ]:
a[1:].mean()

Yikes! That's some wide variation on the acceleration estimates. Our average measurement for the acceleration of gravity is not great, but it's not far off… In case you don't remember, the actual value is $9.8\rm{m/s}^2$.

## Projectile motion

Now, we'll study projectile motion, using a video of a ball "fired" horizontally, like a projectile. Here's a neat video we found online, produced by the folks over at [Flipping Physics](http://www.flippingphysics.com).

In [ ]:
from IPython.display import YouTubeVideo
vid = YouTubeVideo("Y4jgJK35Gf0")
display(vid)

We'd like to capture the coordinates of mouse clicks on a _sequence_ of images, so that we may have the positions of the moving ball caught on video. We know how to capture the coordinates of mouse clicks, so the challenge is to get consecutive frames of the video displayed for us, to click on the ball position each time. 

Widgets to the rescue! There are currently [10 different widget types](http://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html) included in the `ipywidgets` library. The `BoundedIntText()` widget shows a text box with an integer value that can be stepped from a minimum to a maximum value by clicking up/down arrows. Stepping through frames with this widget, and clicking on the ball position each time, gets us what we want.

Digitizing the ball positions in this way is a bit tedious. But this could be a realistic scenario: you captured video of a moving object, and you need to get position data from the video frames. Unless you have some fancy motion-capture equipment, this will do the trick.

Let's load the Jupyter widgets:

In [ ]:
from ipywidgets import widgets

Now, we'll load the video, previously converted to .mp4 format to be read by `imageio`. Notice that it has 3531 frames, and they are 720x1280 pixels in size. 

Below, we're showing frame number 52, which we found to be the start of the portion shown at 50% speed. Use that frame to capture mouse clicks on the intersection of several $10\rm{cm}$ lines, so you can calculate the scaling from pixels to physical distance.

In [ ]:
reader = imageio.get_reader('Projectile_Motion.mp4')

In [ ]:
len(reader)

In [ ]:
image = reader.get_data(52)
image.shape

In [ ]:
fig = pyplot.figure()
pyplot.imshow(image, interpolation='nearest')

coords2 = []
def onclick(event):
    '''Capture the x,y coordinates of a mouse click on the image'''
    ix, iy = event.xdata, event.ydata
    coords2.append([ix, iy]) 

connectId = fig.canvas.mpl_connect('button_press_event', onclick)


In [ ]:
coords2

In [ ]:
y_lines2 = numpy.array(coords2)[:,1]
y_lines2

In [ ]:
gap_lines2 = y_lines2[1:] - y_lines2[0:-1]
gap_lines2.mean()

Above, we repeated the process to compute the vertical distance between the $10\rm{cm}$ marks (averaging over our clicks): the scaling of distances from this video will need multiplying by $0.1$ to get meters, and dividing by `gap_lines2.mean()`.

Now the fun part! Study the code below: we create a `selector` widget of the `BoundedIntText` type, taking the values from 52 to 77, and stepping by 1. We already played around a lot with the video and found this frame range to contain the portion shown at 50% speed. 

We re-use the `onclick()` function, this time appending to a list named `coords3`, and we call it with an event connection from Matplotlib, just like before. But now we add a call to `widgets.interact()`, using a new function named `catchclick()` that reads a new video frame and refreshes the figure with it.

Execute this cell, then click on the ball position, advance a frame, click on the new ball position, and so on, until frame 77. The mouse click positions will be saved in `coords3`.

We found it better to click on the bottom edge of the ball image, rather than attempt to aim at the ball's center.

In [ ]:
selector = widgets.BoundedIntText(value=52, min=52, max=77, step=1,
    description='Frame:',
    disabled=False)

coords3 = []
def onclick(event):
    '''Capture the x,y coordinates of a mouse click on the image'''
    ix, iy = event.xdata, event.ydata
    coords3.append([ix, iy]) 


def catchclick(frame):
    image = reader.get_data(frame)
    pyplot.imshow(image, interpolation='nearest');



fig = pyplot.figure()

connectId = fig.canvas.mpl_connect('button_press_event', onclick)

widgets.interact(catchclick, frame=selector);

In [ ]:
coords3

Now, convert the positions to meters, using our scaling for this video, and save the $x$ and $y$ coordinats to new arrays. Below, we plot the ball positions that we captured.

In [ ]:
x = numpy.array(coords3)[:-2,0] *0.1 / gap_lines2.mean()
y = numpy.array(coords3)[:-2,1] *0.1 / gap_lines2.mean()

In [ ]:
fig = pyplot.figure()
pyplot.scatter(x,-y);

Now, compute the vertical displacements, then get the vertical velocity and acceleration. And repeat the process for the horizontal direction of motion.

In [ ]:
delta_y = (y[1:] - y[:-1])

In [ ]:
vy = delta_y * 60
ay = (vy[1:] - vy[:-1]) * 60
print('The acceleration in the y direction is: {:.2f}'.format(ay.mean()))

In [ ]:
delta_x = (x[1:] - x[:-1])
vx = delta_x * 60
ax = (vx[1:] - vx[:-1]) * 60
print('The acceleration in the x direction is: {:.2f}'.format(ax.mean()))

##### Discuss

* What did you get for the $x$ and $y$ accelerations? What did your neighbor get?
* Do the results make sense to you? Why or why not?

## References

1.  Strobe of a Falling Ball (2008), MIT Department of Physics Technical Services Group, video under CC-BY-NC, available online on [MIT TechTV](http://techtv.mit.edu/collections/physicsdemos/videos/831-strobe-of-a-falling-ball).

2. The Classic Bullet Projectile Motion Experiment with X & Y Axis Scales (2004), video by [Flipping Physics](http://www.flippingphysics.com/bullet-with-scales.html), Jon Thomas-Palmer. Used with permission.

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../../style/custom.css'
HTML(open(css_file, "r").read())